In [97]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import pickle

In [124]:
class NNet(nn.Module):

    def __init__(self,input_dim,output_dim,num_hidden_layers, neurons_per_layer, dropout_rate):
        
        super().__init__()

        layers = []

        for i in range(num_hidden_layers):

            layers.append(nn.Linear(input_dim, neurons_per_layer))
            layers.append(nn.BatchNorm1d(neurons_per_layer))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
            input_dim = neurons_per_layer

        layers.append(nn.Linear(neurons_per_layer, output_dim))
        layers.append(nn.Sigmoid())
        
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [125]:
model = NNet(12,1,2,32,0.5)
model.load_state_dict(torch.load("model_complete.pth"))

<All keys matched successfully>

In [126]:
with open("label_encoding_gender.pkl","rb") as file:
    le = pickle.load(file)

with open("one_hot_encoding_geography.pkl","rb") as file:
    ohe = pickle.load(file)

with open("standar_scalar.pkl","rb") as file:
    ss = pickle.load(file)

In [127]:
input_data = {
        "CreditScore":600,
        "Geography":"France",
        "Gender":"Male",
        "Age":40,
        "Tenure":3,
        "Balance":60000,
        "NumOfProducts":2,
        "HasCrCard": 1,
        "IsActiveMember":1,
        "EstimatedSalary": 50000
}

In [128]:
input_df = pd.DataFrame([input_data])

In [129]:
input_df["Gender"] = le.transform(input_df['Gender'])

In [130]:
geo_data = ohe.transform(input_df[["Geography"]])

In [131]:
geo_columns = ohe.get_feature_names_out(["Geography"])
geography_df = pd.DataFrame(geo_data.toarray(), columns=geo_columns)


In [132]:
geo_columns
geography_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [133]:
final_df = pd.concat([geography_df,input_df.drop(["Geography"],axis=1)],axis=1)
final_df

,Geography_France,Geography_Germany,Geography_Spain,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,1.0,0.0,0.0,600,1,40,3,60000,2,1,1,50000


In [ ]:
from torch import tensor

x_test = ss.transform(final_df)
x_test_torch = tensor(x_test,dtype=torch.float32)

In [143]:
model.eval()
with torch.no_grad():
    prediction = model(x_test_torch).round()

In [144]:
prediction

tensor([[0.]])